# exploring 2017 test and training sets

In [117]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

In [2]:
train = pd.read_csv('BPI Challenge 2017-training.csv')
test = pd.read_csv('BPI Challenge 2017-test.csv')

In [3]:
test_time = test['event time:timestamp'].copy()
train_time = train['event time:timestamp'].copy()

In [4]:
train.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,0,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,A_Create Application,Application,Application_652823628,complete,01-01-2016 10:51:15.304
1,1,Existing loan takeover,New credit,Application_652823628,20000.0,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,01-01-2016 10:51:15.352
2,2,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,01-01-2016 10:51:15.774
3,3,Existing loan takeover,New credit,Application_652823628,20000.0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,01-01-2016 10:52:36.392
4,4,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,01-01-2016 10:52:36.403


In [5]:
test.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,108263240630272,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,A_Create Application,Application,Application_424064154,complete,19-10-2016 01:42:39.265
1,108263240630273,Existing loan takeover,New credit,Application_424064154,17250.0,statechange,User_1,A_Submitted,Application,ApplState_875814280,complete,19-10-2016 01:42:40.100
2,108263240630274,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,W_Handle leads,Workflow,Workitem_1964338518,schedule,19-10-2016 01:42:40.711
3,108267535597568,"Other, see explanation",New credit,Application_608775916,10000.0,Created,User_1,A_Create Application,Application,Application_608775916,complete,19-10-2016 04:16:17.680
4,108267535597569,"Other, see explanation",New credit,Application_608775916,10000.0,statechange,User_1,A_Submitted,Application,ApplState_1860095003,complete,19-10-2016 04:16:18.446


In [6]:
#dt_test = pd.to_datetime(test_time)
#dt_train = pd.to_datetime(train_time)

In [7]:
# dt_test

In [8]:
# dt_train

# Set 2012

In [82]:
train = pd.read_csv('BPI_2012_train.csv')
test = pd.read_csv('BPI_2012_test.csv')

In [3]:
train.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256


In [12]:
cases = list(train.case_name.unique())

In [13]:
# dicts={}

# for case in cases:
    
#     ran = len(train[train['case_name']==case])
#     ID = train[['event concept:name']][train['case_name']==case]
    
#     for i in range(ran):
#         dicts[i]= ID.iloc[i][0]

# Method github

In [241]:
df_sort = train.set_index(train.groupby('case concept:name').cumcount(), append = True)

In [9]:
df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
df_new = df_new.to_frame()
df_new = df_new.rename(columns = {'event concept:name' : "common activity"})
df_sort = df_sort.join(df_new["common activity"])

# Explore time differences

In [243]:
df_sort = df_sort.rename(columns = {'event time:timestamp' : "timestamp"})
df_sort['timestamp'] = pd.to_datetime(df_sort.timestamp)

In [244]:
df_times = df_sort.groupby('case concept:name')["timestamp"].transform(lambda x: x.diff())
df_times = df_times.to_frame()
df_times = df_times.rename(columns = {'timestamp' : "time_diff"})

In [245]:
df_sort = df_sort.join(df_times["time_diff"])

In [246]:
df_sort.head()

,,Unnamed: 0,Unnamed: 0.1,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp,time_diff
0,0,0,0,0.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,NaT
1,1,1,1,1.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,00:00:00.334000
2,2,2,2,2.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,00:00:53.026000
3,3,3,3,3.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,00:00:00.969000
4,0,4,4,4.294967e+09,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,2011-01-10 08:08:58.256,NaT


In [247]:
df_sort = df_sort.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [297]:
df_sort.head()

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp,time_diff
0,0,0.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,NaT
1,1,1.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,00:00:00.334000
2,2,2.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,00:00:53.026000
3,3,3.000000e+00,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,00:00:00.969000
4,0,4.294967e+09,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,2011-01-10 08:08:58.256,NaT


In [293]:
df_try = df_sort[["case concept:name", "time_diff"]].copy()

In [300]:
df_try.head()

,,case concept:name,time_diff
0,0,173688,NaT
1,1,173688,00:00:00.334000
2,2,173688,00:00:53.026000
3,3,173688,00:00:00.969000
4,0,173691,NaT


In [209]:
# nishad's part
average=[]
times =[]
c=1



while c < 12:
    for i, df in df_try.groupby('case concept:name'):

        diff = df['time_diff'].iloc[c]
        diff = diff.total_seconds()
        times.append(diff)




    l = len(times)
    summ = times[0]

    for time in times:
        summ+=time



    avg = summ/l
    average.append(avg)
    avg=0
    c+=1

IndexError: single positional indexer is out-of-bounds

In [187]:
# datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
# for n in range(len)
average=[]
times =[]
c=1
d=1
for i, df in df_try.groupby('case concept:name'):
    times =[]
    print(len(i))
    while d < length:
        print(d)
        diff = df['time_diff'].iloc[c] 
        diff = diff.total_seconds()
        times.append(diff)
        print(times)
        d+=1
        
    l = len(times)
    summ = times[0]

    for time in times:
        summ+=time
    
    avg = summ/l
    average.append(avg)
    avg=0
    c+=1
#     print(len(df))

TypeError: object of type 'int' has no len()

In [305]:
times=[]
c=1
for i, df in df_sort.groupby('case concept:name'):
    if len(df)>=c:
        diff = df['time_diff'].iloc[c] 
        diff = diff.total_seconds()
        times.append(diff)
        
l = len(times)
summ = times[0]

for time in times:
    summ+=time
    
avg = summ/l
avg

0.5826723660330494

In [306]:
times

[0.334,
 3.939,
 0.304,
 0.169,
 0.208,
 0.175,
 0.089,
 0.188,
 0.145,
 0.288,
 0.294,
 0.649,
 0.271,
 0.14,
 0.264,
 0.093,
 0.239,
 0.202,
 0.169,
 0.19,
 0.219,
 0.256,
 1.321,
 0.202,
 0.331,
 0.625,
 0.099,
 0.133,
 0.1,
 0.11,
 0.173,
 0.114,
 0.134,
 0.162,
 0.145,
 0.177,
 0.493,
 0.127,
 0.157,
 0.205,
 0.129,
 0.435,
 0.13,
 0.649,
 0.174,
 1.041,
 0.345,
 0.145,
 0.186,
 0.323,
 0.09,
 0.136,
 0.165,
 0.204,
 0.225,
 0.092,
 0.169,
 0.171,
 0.168,
 0.091,
 0.145,
 1.041,
 0.444,
 0.078,
 0.198,
 0.173,
 0.095,
 1.984,
 0.389,
 0.155,
 0.192,
 0.161,
 0.205,
 0.152,
 0.197,
 0.131,
 0.412,
 0.238,
 0.232,
 0.081,
 0.225,
 0.152,
 0.277,
 0.145,
 0.088,
 0.175,
 0.193,
 0.266,
 0.154,
 0.228,
 0.185,
 0.103,
 0.211,
 2.93,
 1.67,
 0.387,
 0.406,
 0.338,
 0.874,
 0.359,
 0.294,
 0.247,
 0.237,
 0.467,
 0.33,
 0.241,
 0.497,
 0.405,
 0.323,
 0.345,
 0.365,
 0.358,
 0.348,
 0.372,
 0.434,
 0.425,
 0.458,
 0.279,
 0.348,
 0.344,
 0.553,
 0.177,
 0.389,
 0.417,
 0.251,
 0.222,
 8

In [307]:
c=1
for i, df in df_try.groupby('case concept:name'):
    if len(df)>=c:
        print(df['time_diff'].iloc[c])
#         print(df)

0 days 00:00:00.334000
0 days 00:00:03.939000
0 days 00:00:00.304000
0 days 00:00:00.169000
0 days 00:00:00.208000
0 days 00:00:00.175000
0 days 00:00:00.089000
0 days 00:00:00.188000
0 days 00:00:00.145000
0 days 00:00:00.288000
0 days 00:00:00.294000
0 days 00:00:00.649000
0 days 00:00:00.271000
0 days 00:00:00.140000
0 days 00:00:00.264000
0 days 00:00:00.093000
0 days 00:00:00.239000
0 days 00:00:00.202000
0 days 00:00:00.169000
0 days 00:00:00.190000
0 days 00:00:00.219000
0 days 00:00:00.256000
0 days 00:00:01.321000
0 days 00:00:00.202000
0 days 00:00:00.331000
0 days 00:00:00.625000
0 days 00:00:00.099000
0 days 00:00:00.133000
0 days 00:00:00.100000
0 days 00:00:00.110000
0 days 00:00:00.173000
0 days 00:00:00.114000
0 days 00:00:00.134000
0 days 00:00:00.162000
0 days 00:00:00.145000
0 days 00:00:00.177000
0 days 00:00:00.493000
0 days 00:00:00.127000
0 days 00:00:00.157000
0 days 00:00:00.205000
0 days 00:00:00.129000
0 days 00:00:00.435000
0 days 00:00:00.130000
0 days 00:0

0 days 00:00:00.182000
0 days 00:00:00.101000
0 days 00:00:03.004000
0 days 00:00:00.205000
0 days 00:00:00.131000
0 days 00:00:00.166000
0 days 00:00:00.298000
0 days 00:00:00.085000
0 days 00:00:00.297000
0 days 00:00:00.092000
0 days 00:00:00.233000
0 days 00:00:00.087000
0 days 00:00:00.147000
0 days 00:00:00.180000
0 days 00:00:00.126000
0 days 00:00:00.227000
0 days 00:00:00.098000
0 days 00:00:00.099000
0 days 00:00:00.554000
0 days 00:00:00.178000
0 days 00:00:00.151000
0 days 00:00:00.141000
0 days 00:00:00.278000
0 days 00:00:00.257000
0 days 00:00:00.090000
0 days 00:00:06.667000
0 days 00:00:00.172000
0 days 00:00:02.457000
0 days 00:00:01.269000
0 days 00:00:00.134000
0 days 00:00:01.572000
0 days 00:00:00.319000
0 days 00:00:01.558000
0 days 00:00:00.263000
0 days 00:00:00.279000
0 days 00:00:00.127000
0 days 00:00:01.018000
0 days 00:00:00.790000
0 days 00:00:00.087000
0 days 00:00:00.137000
0 days 00:00:00.253000
0 days 00:00:00.541000
0 days 00:00:00.158000
0 days 00:0

0 days 00:00:00.224000
0 days 00:00:00.142000
0 days 00:00:00.253000
0 days 00:00:00.238000
0 days 00:00:00.168000
0 days 00:00:00.194000
0 days 00:00:00.191000
0 days 00:00:00.157000
0 days 00:00:00.240000
0 days 00:00:00.144000
0 days 00:00:00.176000
0 days 00:00:00.479000
0 days 00:00:00.207000
0 days 00:00:00.230000
0 days 00:00:00.272000
0 days 00:00:00.177000
0 days 00:00:00.173000
0 days 00:00:00.189000
0 days 00:00:00.174000
0 days 00:00:07.730000
0 days 00:00:07.460000
0 days 00:00:00.206000
0 days 00:00:00.991000
0 days 00:00:00.124000
0 days 00:00:00.210000
0 days 00:00:00.092000
0 days 00:00:00.101000
0 days 00:00:00.531000
0 days 00:00:00.627000
0 days 00:00:00.878000
0 days 00:00:00.541000
0 days 00:00:00.189000
0 days 00:00:00.135000
0 days 00:00:00.104000
0 days 00:00:00.133000
0 days 00:00:00.177000
0 days 00:00:00.144000
0 days 00:00:00.236000
0 days 00:00:00.106000
0 days 00:00:00.220000
0 days 00:00:00.292000
0 days 00:00:00.308000
0 days 00:00:00.211000
0 days 00:0

0 days 00:00:00.255000
0 days 00:00:00.266000
0 days 00:00:00.094000
0 days 00:00:00.238000
0 days 00:00:00.187000
0 days 00:00:00.111000
0 days 00:00:00.110000
0 days 00:00:00.337000
0 days 00:00:00.158000
0 days 00:00:00.179000
0 days 00:00:00.102000
0 days 00:00:00.239000
0 days 00:00:00.213000
0 days 00:00:00.147000
0 days 00:00:00.084000
0 days 00:00:00.232000
0 days 00:00:00.090000
0 days 00:00:00.190000
0 days 00:00:00.199000
0 days 00:00:00.228000
0 days 00:00:00.151000
0 days 00:00:00.154000
0 days 00:00:00.098000
0 days 00:00:00.377000
0 days 00:00:00.160000
0 days 00:00:00.072000
0 days 00:00:00.245000
0 days 00:00:00.333000
0 days 00:00:00.202000
0 days 00:00:00.156000
0 days 00:00:00.178000
0 days 00:00:00.071000
0 days 00:00:00.264000
0 days 00:00:01.695000
0 days 00:00:00.107000
0 days 00:00:00.089000
0 days 00:00:00.852000
0 days 00:00:00.137000
0 days 00:00:02.372000
0 days 00:00:00.558000
0 days 00:00:00.423000
0 days 00:00:00.514000
0 days 00:00:00.455000
0 days 00:0

0 days 00:00:00.843000
0 days 00:00:00.104000
0 days 00:00:00.365000
0 days 00:00:00.110000
0 days 00:00:00.253000
0 days 00:00:00.246000
0 days 00:00:00.234000
0 days 00:00:00.341000
0 days 00:00:00.106000
0 days 00:00:00.135000
0 days 00:00:00.111000
0 days 00:00:00.726000
0 days 00:00:00.100000
0 days 00:00:00.156000
0 days 00:00:00.120000
0 days 00:00:00.133000
0 days 00:00:00.343000
0 days 00:00:00.195000
0 days 00:00:00.387000
0 days 00:00:00.315000
0 days 00:00:00.165000
0 days 00:00:00.121000
0 days 00:00:00.196000
0 days 00:00:00.248000
0 days 00:00:00.145000
0 days 00:00:00.866000
0 days 00:00:00.634000
0 days 00:00:00.153000
0 days 00:00:00.190000
0 days 00:00:00.226000
0 days 00:00:00.250000
0 days 00:00:00.173000
0 days 00:00:00.159000
0 days 00:00:00.155000
0 days 00:00:00.110000
0 days 00:00:00.173000
0 days 00:00:00.674000
0 days 00:00:00.371000
0 days 00:00:00.090000
0 days 00:00:00.159000
0 days 00:00:00.204000
0 days 00:00:01.280000
0 days 00:00:00.470000
0 days 00:0

...

0 days 00:00:04.625000
0 days 00:00:00.272000
0 days 00:00:00.390000
0 days 00:00:00.746000
0 days 00:00:00.517000
0 days 00:00:00.337000
0 days 00:00:00.387000
0 days 00:00:00.252000
0 days 00:00:01.238000
0 days 00:00:00.586000
0 days 00:00:00.417000
0 days 00:00:00.272000
0 days 00:00:00.339000
0 days 00:00:00.490000
0 days 00:00:00.467000
0 days 00:00:02.140000
0 days 00:00:00.281000
0 days 00:00:01.807000
0 days 00:00:00.514000
0 days 00:00:00.286000
0 days 00:00:00.406000
0 days 00:00:00.417000
0 days 00:00:00.368000
0 days 00:00:00.254000
0 days 00:00:00.214000
0 days 00:00:00.524000
0 days 00:00:00.516000
0 days 00:00:00.273000
0 days 00:00:00.408000
0 days 00:00:00.374000
0 days 00:00:00.373000
0 days 00:00:01.153000
0 days 00:00:00.646000
0 days 00:00:00.577000
0 days 00:00:00.381000
0 days 00:00:00.403000
0 days 00:00:00.638000
0 days 00:00:00.521000
0 days 00:00:00.325000
0 days 00:00:00.304000
0 days 00:00:00.473000
0 days 00:00:00.927000
0 days 00:00:00.409000
0 days 00:0

0 days 00:00:01.118000
0 days 00:00:09.228000
0 days 00:00:01.582000
0 days 00:00:00.323000
0 days 00:00:00.134000
0 days 00:00:00.867000
0 days 00:00:00.434000
0 days 00:00:01.424000
0 days 00:00:00.375000
0 days 00:00:01.376000
0 days 00:00:01.528000
0 days 00:00:01.323000
0 days 00:00:00.213000
0 days 00:00:00.346000
0 days 00:00:00.360000
0 days 00:00:00.235000
0 days 00:00:00.136000
0 days 00:00:00.648000
0 days 00:00:01.112000
0 days 00:00:00.215000
0 days 00:00:00.159000
0 days 00:00:00.156000
0 days 00:00:01.035000
0 days 00:00:00.111000
0 days 00:00:00.448000
0 days 00:00:00.150000
0 days 00:00:00.172000
0 days 00:00:00.368000
0 days 00:00:00.639000
0 days 00:00:00.151000
0 days 00:00:00.130000
0 days 00:00:00.517000
0 days 00:00:00.123000
0 days 00:00:01.052000
0 days 00:00:01.280000
0 days 00:00:00.139000
0 days 00:00:01.194000
0 days 00:00:00.235000
0 days 00:00:00.246000
0 days 00:00:00.120000
0 days 00:00:00.150000
0 days 00:00:00.703000
0 days 00:00:00.256000
0 days 00:0

0 days 00:00:00.136000
0 days 00:00:00.187000
0 days 00:00:00.126000
0 days 00:00:00.104000
0 days 00:00:00.360000
0 days 00:00:00.183000
0 days 00:00:00.397000
0 days 00:00:01.507000
0 days 00:00:00.171000
0 days 00:00:00.114000
0 days 00:00:01.270000
0 days 00:00:00.151000
0 days 00:00:00.217000
0 days 00:00:00.238000
0 days 00:00:00.528000
0 days 00:00:00.330000
0 days 00:00:00.543000
0 days 00:00:00.529000
0 days 00:00:00.100000
0 days 00:00:00.165000
0 days 00:00:01.668000
0 days 00:00:00.155000
0 days 00:00:00.298000
0 days 00:00:00.187000
0 days 00:00:00.258000
0 days 00:00:00.185000
0 days 00:00:03.513000
0 days 00:00:00.194000
0 days 00:00:00.403000
0 days 00:00:00.181000
0 days 00:00:00.209000
0 days 00:00:00.534000
0 days 00:00:00.153000
0 days 00:00:00.135000
0 days 00:00:00.748000
0 days 00:00:00.175000
0 days 00:00:00.242000
0 days 00:00:00.107000
0 days 00:00:00.160000
0 days 00:00:00.118000
0 days 00:00:00.082000
0 days 00:00:00.232000
0 days 00:00:00.095000
0 days 00:0

0 days 00:00:00.139000
0 days 00:00:00.442000
0 days 00:00:00.335000
0 days 00:00:00.157000
0 days 00:00:00.176000
0 days 00:00:00.485000
0 days 00:00:00.145000
0 days 00:00:00.170000
0 days 00:00:00.197000
0 days 00:00:00.352000
0 days 00:00:00.274000
0 days 00:00:00.110000
0 days 00:00:00.304000
0 days 00:00:00.113000
0 days 00:00:00.723000
0 days 00:00:00.126000
0 days 00:00:02.415000
0 days 00:00:00.823000
0 days 00:00:00.161000
0 days 00:00:01.247000
0 days 00:00:00.182000
0 days 00:00:00.363000
0 days 00:00:00.236000
0 days 00:00:00.164000
0 days 00:00:00.111000
0 days 00:00:00.096000
0 days 00:00:00.298000
0 days 00:00:00.244000
0 days 00:00:00.482000
0 days 00:00:00.209000
0 days 00:00:00.113000
0 days 00:00:00.239000
0 days 00:00:00.208000
0 days 00:00:00.586000
0 days 00:00:00.163000
0 days 00:00:00.089000
0 days 00:00:00.251000
0 days 00:00:00.868000
0 days 00:00:00.690000
0 days 00:00:00.152000
0 days 00:00:00.161000
0 days 00:00:00.308000
0 days 00:00:00.252000
0 days 00:0

0 days 00:00:05.355000
0 days 00:00:00.334000
0 days 00:00:00.333000
0 days 00:00:00.149000
0 days 00:00:00.830000
0 days 00:00:00.116000
0 days 00:00:00.231000
0 days 00:00:00.434000
0 days 00:00:00.178000
0 days 00:00:00.806000
0 days 00:00:01.386000
0 days 00:00:00.166000
0 days 00:00:03.185000
0 days 00:00:01.099000
0 days 00:00:00.179000
0 days 00:00:00.266000
0 days 00:00:00.329000
0 days 00:00:00.288000
0 days 00:00:03.110000
0 days 00:00:00.173000
0 days 00:00:00.155000
0 days 00:00:00.149000
0 days 00:00:00.188000
0 days 00:00:00.111000
0 days 00:00:00.139000
0 days 00:00:01.012000
0 days 00:00:00.170000
0 days 00:00:00.162000
0 days 00:00:00.333000
0 days 00:00:00.860000
0 days 00:00:00.562000
0 days 00:00:00.281000
0 days 00:00:00.184000
0 days 00:00:00.214000
0 days 00:00:00.515000
0 days 00:00:01.319000
0 days 00:00:00.185000
0 days 00:00:01.352000
0 days 00:00:00.130000
0 days 00:00:00.228000
0 days 00:00:01.131000
0 days 00:00:00.354000
0 days 00:00:02.334000
0 days 00:0

In [308]:
df_try[:10]

,,case concept:name,time_diff
0,0,173688,NaT
1,1,173688,00:00:00.334000
2,2,173688,00:00:53.026000
3,3,173688,00:00:00.969000
4,0,173691,NaT
5,1,173691,00:00:03.939000
6,2,173691,00:00:54.453000
7,3,173691,00:00:02.930000
8,0,173694,NaT
9,1,173694,00:00:00.304000


# Function for adding the next event

In [2]:
def make_event_pred(df_train, df_test):
    
    '''Parameters: df_train: training set
                   df_test: test set
                   
       We can change to csv as input and making the sets or create a different function for it'''
    preds = pd.DataFrame() # we can return this df in order to et only the predictions for accuracy scores
    
    df_sort = df_train.set_index(df_train.groupby('case concept:name').cumcount(), append = True)
    df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
    df_new = df_new.to_frame()
    common_acts = df_new.rename(columns = {'event concept:name' : "common activity"})
    
    test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)
    
    # Here we can implement the average time to add later in the dataframe
    test = test.rename(columns = {'event time:timestamp' : "timestamp"})
    test['timestamp'] = pd.to_datetime(test.timestamp) 
    
    
    max_list = common_acts.groupby(level=1).apply(max).copy() # Get process with max(N) for predictions
    
    for i, df in test.groupby('case concept:name'):
        length = len(df)
        test = test.append({'case concept:name':int(i),
                            'event concept:name': max_list.loc[length, 'common activity']}, ignore_index=True)
                            #'timestamp': dt.datetime(2012, 2, 14, 23, 59, 59)}, ignore_index=True)
                            # here we can add the time of the added new event
                
        preds = preds.append({'case concept:name':int(i),
                            'event concept:name': max_list.loc[length, 'common activity']}, ignore_index=True)
                            #'timestamp': dt.datetime(2012, 2, 14, 23, 59, 59)}, ignore_index=True) 
            
    test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
    
    # Depending on how we want it we should sort: test = test.sort_values(time, case name, etc.)
    test = test.sort_values('timestamp') # now sorted by time
    
    return preds #test

# Test stuff for the function

In [148]:
pos = df_new.groupby(level=1).apply(max).copy()

In [214]:
df_test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
df_test.tail()

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
47818,35,54666343743523,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,START,14-03-2012 15:59:28.309
47819,36,54666343743524,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:00:09.680
47820,32,49495203119136,209595,2012-02-15T10:10:36.503+01:00,13000,W_Nabellen offertes,START,14-03-2012 16:02:03.883
47821,34,52342766436386,211624,2012-02-21T23:38:40.044+01:00,35000,W_Nabellen incomplete dossiers,START,14-03-2012 16:04:46.192
47822,35,52342766436387,211624,2012-02-21T23:38:40.044+01:00,35000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:04:54.681


In [215]:
df_test = df_test.rename(columns = {'event time:timestamp' : "timestamp"})
df_test['timestamp'] = pd.to_datetime(df_test.timestamp)

for i, df in df_test.groupby('case concept:name'):
    length = len(df)
    df_test = df_test.append({'case concept:name':int(i),
                              'event concept:name': pos.loc[length, 'common activity'],
                              'timestamp': dt.datetime(2012, 12, 30, 23, 59, 59)},ignore_index=True)
df_test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)

In [216]:
df_test[df_test['case concept:name']==206324]

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp
0,0,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_SUBMITTED,COMPLETE,2012-03-02 17:17:11.047
1,1,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:17:11.323
2,2,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_DECLINED,COMPLETE,2012-03-02 17:17:42.964
47823,3,NaN,206324,NaN,NaN,W_Afhandelen leads,NaN,2010-12-30 23:59:59.000


In [219]:
df_test.sort_values('timestamp')

# Testing function

In [3]:
train = pd.read_csv('BPI_2012_train.csv')
test = pd.read_csv('BPI_2012_test.csv')

In [4]:
saved = make_event_pred(train, test)

In [5]:
saved[saved['case concept:name']==206327]

,case concept:name,event concept:name
351,206327.0,W_Completeren aanvraag


In [6]:
saved

,case concept:name,event concept:name
0,188485.0,W_Afhandelen leads
1,189805.0,W_Nabellen offertes
2,191644.0,W_Completeren aanvraag
3,193647.0,W_Completeren aanvraag
4,194139.0,W_Completeren aanvraag
...,...,...
2963,214364.0,W_Nabellen offertes
2964,214367.0,W_Afhandelen leads
2965,214370.0,W_Completeren aanvraag
2966,214373.0,W_Nabellen offertes
